# Testing the LZW compression code.
Stough
DIP 8.2.4 on LZW encoding

- [Duke site on LZW](https://www2.cs.duke.edu/csed/curious/compression/lzw.html)
- [Nice demo codes](http://rosettacode.org/wiki/LZW_compression#Python)

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

from LZWUtil import *
from LZWUtilNeg import *

import time



import scipy.ndimage as ndimage
from scipy.ndimage import correlate
from skimage.filters import *
from scipy.stats import entropy



BITS = 12

In [2]:
I = loadHuffableImage('skyandsea.jpg')
I1d = I.ravel()

starttime = time.perf_counter()

estream = LZWCompressNeg(I1d, bits=BITS) #NEG


encodedSize = sum(1 for x in estream)

endtime = time.perf_counter()

print('I original size %6.3fKB, LZ encoded size %6.3fKB.' %
      (I.size/1024, encodedSize*BITS/(8*1024)))

print('Encoding time takes %10.4f.' % (endtime - starttime))


starttime = time.perf_counter()

decomp = LZWDecompressNeg(estream, bits=BITS) #NEG

recI1d = np.zeros(I1d.shape)

for i, x in enumerate(decomp):
    recI1d[i] = x
print(i)
endtime = time.perf_counter()


print('Decoding time takes %10.4f.' % (endtime - starttime))


Ir = recI1d.reshape(I.shape)

f, ax = plt.subplots(1,2, figsize=(10,3), sharex=True, sharey=True)
ax[0].imshow(I, cmap='gray')
ax[0].set_title('Original %6.3fKB' % (I.size/1024))

ax[1].imshow(Ir, cmap='gray')
ax[1].set_title('LZ Compressed %6.3fKB' % (encodedSize*BITS/(8*1024)))


print('At this point you can even view the table that '
      'encoding produced. Try estream.table.')


#
# I = [0 0 1 2 3 0 0 2 3 0 0 1 2 3]
# estream = LZWCompress(I)

loadHuffableImage: input is multi-channel, using grayscale.
I original size 1024.500KB, LZ encoded size 963.001KB.
Encoding time takes    27.3695.
1049087
Decoding time takes    30.9668.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

At this point you can even view the table that encoding produced. Try estream.table.


In [3]:
#estream.instream.min()


In [4]:
#plt.figure()
#plt.imshow(plt.imread("bacon.png"))


In [5]:
def arrInfo(arr):
    return arr.shape, arr.min(), arr.max(), arr.dtype


I = plt.imread("bacon.png")#.astype(np.int16)
arrInfo(I[...,:2]*255)

((162, 788, 2), 13.0, 255.0, dtype('float32'))

In [6]:
I = plt.imread("thanos.jpeg").astype(np.int16)

#using the pixel above by transpposing the picture then re transpoing
T = I.copy()
T = T.T
h = np.array([-1, 1], ndmin=2).astype(np.int16)
ItR = correlate(I.T[0,...].astype(np.int16), h, mode='constant', cval=0).T
ItB = correlate(I.T[1,...].astype(np.int16), h, mode='constant', cval=0).T
ItG = correlate(I.T[2,...].astype(np.int16), h, mode='constant', cval=0).T
It = np.stack((ItR,ItB,ItG),axis=-1)




h = np.array([-1, 1], ndmin=2).astype(np.int16)
IfR = correlate(I[...,0].astype(np.int16), h, mode='constant', cval=0)
IfB = correlate(I[...,1].astype(np.int16), h, mode='constant', cval=0)
IfG = correlate(I[...,2].astype(np.int16), h, mode='constant', cval=0)
If = np.stack((IfR,IfB,IfG),axis=-1)



h = np.array([1, -1], ndmin=2).astype(np.int16)
IaR = correlate(I[...,0].astype(np.int16), h, mode='constant', cval=0)
IaB = correlate(I[...,1].astype(np.int16), h, mode='constant', cval=0)
IaG = correlate(I[...,2].astype(np.int16), h, mode='constant', cval=0)
Ia = np.stack((IaR,IaB,IaG),axis=-1)

Iflt = I.copy().astype(float) / 255

h = np.array([.5, .5], ndmin=2).astype(np.float)
IavR = correlate(Iflt[...,0].astype(np.float), h, mode='constant', cval=0)
IavB = correlate(Iflt[...,1].astype(np.float), h, mode='constant', cval=0)
IavG = correlate(Iflt[...,2].astype(np.float), h, mode='constant', cval=0)
Iav = np.stack((IavR,IavB,IavG),axis=-1)


raw_Ia = Ia.copy()
#clipping
Ia = Ia-Ia.min()
Ia[Ia > 255] = 255

If = If-If.min()
If[If > 255] = 255

It = It-It.min()
It[It > 255] = 255



In [7]:
arrInfo(raw_Ia)

((630, 1200, 3), -126, 79, dtype('int16'))

In [8]:
def PlotWithHist(I, title):
    f, ax2 = plt.subplots(1,2, figsize=(10,3))
    I_hist, I_bins = np.histogram(I.ravel(), bins=np.arange(257))
    print("the entorpy of", title, "is",entropy(I_hist, base=2))
    ax2[0].imshow(I) #https://matplotlib.org/api/_as_gen/matplotlib.pyplot.imshow.html
    ax2[0].set_title(title)
    ax2[1].hist(I[...,0].ravel(), 250, alpha = .6, label = 'red', color = 'r')
    ax2[1].hist(I[...,1].ravel(), 250, alpha = .6, label = 'green', color = 'g')
    ax2[1].hist(I[...,2].ravel(), 250, alpha = .6, label = 'blue', color = 'b')
    ax2[1].legend(loc = 'upper right')

In [9]:
PlotWithHist(I,"Original")
PlotWithHist(Ia, "sub")
PlotWithHist(It, "top")
PlotWithHist(Iav, "average")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

the entorpy of Original is 7.676283193723261


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

the entorpy of sub is 3.87646710616502


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

the entorpy of top is 4.01568206521397


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

the entorpy of average is 0.012808536430404943


In [10]:
Iav = (Iav*255).astype(np.uint8)
Iav_hist, Iav_bins = np.histogram(Iav.ravel(), bins=np.arange(257))
print(entropy(Iav_hist, base=2))

7.665050390466607


In [14]:
I = loadHuffableImage(raw_Ia)
I1d = raw_Ia.ravel()

starttime = time.perf_counter()

estream = LZWCompressNeg(I1d, bits=12) #NEG


encodedSize = sum(1 for x in estream)

endtime = time.perf_counter()

print('I original size %6.3fKB, LZ encoded size %6.3fKB.' %
      (I.size/1024, encodedSize*BITS/(8*1024)))

print('Encoding time takes %10.4f.' % (endtime - starttime))


starttime = time.perf_counter()

decomp = LZWDecompressNeg(estream, bits=BITS) #NEG

recI1d = np.zeros(I1d.shape)

for i, x in enumerate(decomp):
    recI1d[i] = x
print(i)
endtime = time.perf_counter()


print('Decoding time takes %10.4f.' % (endtime - starttime))


Ir = recI1d.reshape(I.shape)

f, ax = plt.subplots(1,2, figsize=(10,3), sharex=True, sharey=True)
ax[0].imshow(I, cmap='gray')
ax[0].set_title('Original %6.3fKB' % (I.size/1024))

ax[1].imshow(Ir, cmap='gray')
ax[1].set_title('LZ Compressed %6.3fKB' % (encodedSize*BITS/(8*1024)))


print('At this point you can even view the table that '
      'encoding produced. Try estream.table.')


#
# I = [0 0 1 2 3 0 0 2 3 0 0 1 2 3]
# estream = LZWCompress(I)

loadHuffableImage: input is multi-channel, using grayscale.


KeyError: ''

In [13]:
estream.instream
estream.table

{'ÿ': 0,
 'Ā': 1,
 'ā': 2,
 'Ă': 3,
 'ă': 4,
 'Ą': 5,
 'ą': 6,
 'Ć': 7,
 'ć': 8,
 'Ĉ': 9,
 'ĉ': 10,
 'Ċ': 11,
 'ċ': 12,
 'Č': 13,
 'č': 14,
 'Ď': 15,
 'ď': 16,
 'Đ': 17,
 'đ': 18,
 'Ē': 19,
 'ē': 20,
 'Ĕ': 21,
 'ĕ': 22,
 'Ė': 23,
 'ė': 24,
 'Ę': 25,
 'ę': 26,
 'Ě': 27,
 'ě': 28,
 'Ĝ': 29,
 'ĝ': 30,
 'Ğ': 31,
 'ğ': 32,
 'Ġ': 33,
 'ġ': 34,
 'Ģ': 35,
 'ģ': 36,
 'Ĥ': 37,
 'ĥ': 38,
 'Ħ': 39,
 'ħ': 40,
 'Ĩ': 41,
 'ĩ': 42,
 'Ī': 43,
 'ī': 44,
 'Ĭ': 45,
 'ĭ': 46,
 'Į': 47,
 'į': 48,
 'İ': 49,
 'ı': 50,
 'Ĳ': 51,
 'ĳ': 52,
 'Ĵ': 53,
 'ĵ': 54,
 'Ķ': 55,
 'ķ': 56,
 'ĸ': 57,
 'Ĺ': 58,
 'ĺ': 59,
 'Ļ': 60,
 'ļ': 61,
 'Ľ': 62,
 'ľ': 63,
 'Ŀ': 64,
 'ŀ': 65,
 'Ł': 66,
 'ł': 67,
 'Ń': 68,
 'ń': 69,
 'Ņ': 70,
 'ņ': 71,
 'Ň': 72,
 'ň': 73,
 'ŉ': 74,
 'Ŋ': 75,
 'ŋ': 76,
 'Ō': 77,
 'ō': 78,
 'Ŏ': 79,
 'ŏ': 80,
 'Ő': 81,
 'ő': 82,
 'Œ': 83,
 'œ': 84,
 'Ŕ': 85,
 'ŕ': 86,
 'Ŗ': 87,
 'ŗ': 88,
 'Ř': 89,
 'ř': 90,
 'Ś': 91,
 'ś': 92,
 'Ŝ': 93,
 'ŝ': 94,
 'Ş': 95,
 'ş': 96,
 'Š': 97,
 'š': 98,
 'Ţ': 99,
 'ţ': 100,

In [ ]:
for i in range(-10,12):
    print(i)

In [ ]:
myhash = lambda window: ''.join(chr(x) for x in window)
dict(zip([myhash([x]) for x in range(256+255)], range(256+255)))